In [2]:
!pip install selenium webdriver-manager beautifulsoup4 pandas


In [16]:
pip install html-table-parser-python3


Note: you may need to restart the kernel to use updated packages.


In [28]:
!pip install requests


# Using HTML parser

In [29]:
import urllib.request
from pprint import pprint
from html_table_parser.parser import HTMLTableParser
import pandas as pd
import requests

In [34]:
def url_get_contents(url):
    req = urllib.request.Request(url=url)
    f = urllib.request.urlopen(req)
    return f.read()



In [35]:
url = 'https://www.sharesansar.com/company/sarbtm'


In [36]:
xhtml = url_get_contents(url).decode('utf-8')
# xhtml = url_get_contents(url)


In [37]:
p = HTMLTableParser()
p.feed(xhtml)

In [38]:
print(f"\nNumber of tables found: {len(p.tables)}\n")


Number of tables found: 18



In [39]:
for idx, table in enumerate(p.tables):
    print(f"Table {idx}:\n")
    pprint(table)
    print("-" * 60)

Table 0:

[['Bonus Share', '7.00\n                                                %'],
 ['Cash Dividend', '3.00\n                                                %'],
 ['Year', '2080/2081'],
 ['Book Close Date', '2024-12-27']]
------------------------------------------------------------
Table 1:

[['NO RECORD FOUND']]
------------------------------------------------------------
Table 2:

[['NO RECORD FOUND']]
------------------------------------------------------------
Table 3:

[['Support level (S3)', '839.03'],
 ['Support level (S2)', '853.51'],
 ['Support level (S1)', '866.03'],
 ['Pivot Point (PP)', '880.51'],
 ['Resistance level (R1)', '893.03'],
 ['Resistance level (R2)', '907.51'],
 ['Resistance level (R3)', '920.03']]
------------------------------------------------------------
Table 4:

[['MA5', '855.92'],
 ['Signal', 'BULLISH'],
 ['MA20', '811.96'],
 ['Signal', 'BULLISH'],
 ['MA180', '810.60'],
 ['Signal', 'BULLISH']]
-----------------------------------------------------------

In [40]:
# selected_table_index = 1  
# df = pd.DataFrame(p.tables[selected_table_index])

# Beautifulsoup

In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd

# Setup ChromeDriver
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
# options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(service=service, options=options)

try:
    #target page
    url = "https://www.sharesansar.com/company/sarbtm"
    driver.get(url)

    # Wait explicitly for the table to load (up to 15 seconds)
    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.TAG_NAME, "table"))
    )

   
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Find the table
    table = soup.find("table")
    if not table:
        raise Exception("Table not found!")

        
    rows = []
    for tr in table.find_all("tr"):
        cells = [td.get_text(strip=True) for td in tr.find_all(["th", "td"])]
        if cells:
            rows.append(cells)

    # Convert to DataFrame
    df = pd.DataFrame(rows[1:], columns=rows[0])
    df.to_csv("price_history.csv", index=False)
    print("✅ Data saved to 'price_history.csv'")

except Exception as e:
    print("❌ Error:", e)

finally:
    driver.quit()


✅ Data saved to 'price_history.csv'
